In [ ]:
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: MIT-0
#
# Permission is hereby granted, free of charge, to any person obtaining a copy of this
# software and associated documentation files (the "Software"), to deal in the Software
# without restriction, including without limitation the rights to use, copy, modify,
# merge, publish, distribute, sublicense, and/or sell copies of the Software, and to
# permit persons to whom the Software is furnished to do so.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED,
# INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A
# PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT
# HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION
# OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE
# SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: MIT-0
#
# Permission is hereby granted, free of charge, to any person obtaining a copy of this
# software and associated documentation files (the "Software"), to deal in the Software
# without restriction, including without limitation the rights to use, copy, modify,
# merge, publish, distribute, sublicense, and/or sell copies of the Software, and to
# permit persons to whom the Software is furnished to do so.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED,
# INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A
# PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT
# HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION
# OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE
# SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

# Code samples for a webinar 'Managing AWS IoT Greengrass with AWS SDK for Python'
## Table of contents
Part 1: Preparations  
Part 2: Create a Greengrass Group (create_group)   
Part 3: Greengrass Core  
    - Create a Thing in AWS IoT registry and store certificate / private key   
    - Create an IoT Policy and attach it to the newly created certificate   
    - Create a CoreDefinitionVersion and CoreDefinition   
Part 4: Devices   
    - Create 2 Things in AWS IoT registry and store certificate / private key   
    - For each Thing, create an IoT Policy and attach it to the newly created certificate   
    - Create DeviceDefinitionVersion and DeviceDefinition   
Part 5:  Lambda function   
    - Create a new AWS Lambda function   
    - Create FunctionDefinition and FunctionDefinitionVersion   
Part 6: Subscriptions   
    - Create SubscriptionDefinitionVersion and SubscriptionDefinition   
Part 7: Create a GroupVersion   
Part 8: Create a Greengrass configuration file config.json   
Part 9: Prepare configuration files   
    - Greengrass Core certificates and Greengrass configuration   
    - Devices certificates   
Part 10: Deploy a Greengrass Group   
Part 11: Simulate ingestion from Greengrass aware devices   

# Part 1: Preparations

#### Install the neccessary libraries

In [ ]:
!pip install boto3  PyYAML

### Set configuration parameters

In [ ]:
# The Greengrass Groups created by the cod below will have that prefix in a group name
GROUP_PREFIX = "WebinarGGSDK"
# Names of directories to use when creating a Greengrass IoT Core configuration file
DIRECTORY_CERT = "certs"
DIRECTORY_JSONCONFIG = "config"
DIRECTORY_STATE = "./state"
# Path to template file for config.json
# See details in https://docs.aws.amazon.com/greengrass/latest/developerguide/gg-core.html
FILE_GGCONFIGTEMPLATE = "./configtemplate/config.json.template"
# Path to Amazon root CA, see https://docs.aws.amazon.com/iot/latest/developerguide/server-authentication.html for details
ROOTCAPEM = "root.ca.pem"
!WORKSHOP_ROOT=$(pwd)

# Create a function delivering unique id which will be added to the name of various artifacts created by the code in this notebook. Random number generation is used to be able to  restart most commands in this notebook without running into errors because of duplicate identifiers.
from datetime import datetime
import random
UNIQUE_ID = datetime.today().strftime("%Y%m%d%H%M%S") + "_"
def uniqueid():
    return UNIQUE_ID + str(random.randint(1, 10 ^ 5))


# Setup logging
import logging, sys

logging.basicConfig(
    stream=sys.stdout,
    level=logging.INFO,
    format="%(asctime)s %(levelname)-8s %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
log = logging.getLogger(__name__)


# Set region and account id which will later be used for the creation of permissions based
# on "grant least priviledge" guideline.
import boto3
current_region = boto3.session.Session().region_name
current_account_id = boto3.client("sts").get_caller_identity().get("Account")

log.info("Region={}, Account={}".format(current_region, current_account_id))

### Configuration directory#
The following commands will create a new directory for storing the certificates, keys and configuration files

In [ ]:
config_directory = "config-" + uniqueid()

from build_and_deploy_coffee_monitoring_utilities import mkdir

mkdir(config_directory)

ggcore_config_directory_root = config_directory + "/greengrass_core"
# Create directory to store certificate, private and public keys for Core Thing
ggcore_config_directory_cert = ggcore_config_directory_root + "/" + DIRECTORY_CERT
mkdir(ggcore_config_directory_root)
mkdir(ggcore_config_directory_cert)
# Create directory pointing to Greengrass Core configuration
ggcore_config_directory_jsonconfig = (
    ggcore_config_directory_root + "/" + DIRECTORY_JSONCONFIG
)
mkdir(ggcore_config_directory_jsonconfig)


ggad_config_directory_root = config_directory + "/devices"

# Create directory to store certificate, private and public keys for Core Thing
# Please note: storing private keys on a local file system is generally insecure and is done in 
# this code sample only for demonstrational purposes assuming prototypic usage. It should be only used in 
# prototyping environments after detailed risk analysis and never used in production environments.
ggad_config_directory_cert = ggad_config_directory_root + "/" + DIRECTORY_CERT
mkdir(ggad_config_directory_root)
mkdir(ggad_config_directory_cert)

log.info("Directories created: {}, {}".format(ggcore_config_directory_cert,ggcore_config_directory_jsonconfig))

### Import libraries

In [ ]:
import json
import yaml
import time
import os
import uuid
import boto3
import random
from build_and_deploy_coffee_monitoring_utilities import *

### Create AWS for Python (Boto3) SDK clients for AWS IoT Greengrass, AWS IoT Core and Amazon IAM

In [ ]:
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/greengrass.html
gg_client = boto3.client("greengrass")
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/iot.html
iot_client = boto3.client("iot")
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/iam.html
iam_client = boto3.client("iam")
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/lambda.html
lambda_client = boto3.client("lambda")

# Part 2: Create Group entity and assign a Group role



### Create Group entity

In [ ]:
# Define a name for a Greengrass Group
GROUP_NAME = uniqueid() + "_" + GROUP_PREFIX

# After executing the "create_group" call a Greengrass Group will be visible in an AWS Management Console
group = gg_client.create_group(Name=GROUP_NAME)
group_id = group["Id"]

# Print result of API call
group.pop("ResponseMetadata") and group

### Set a Group Role for the Greengrass Group
A Group Role role contains the permissions that AWS IoT Greengrass features (e.g. logging, AWS Lambda functions) use to access AWS cloud services. 

In [1]:
# The following inline policy will restrict Greengrsass Group permissions to logging activties.
group_role_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {"Effect": "Allow", "Action": "logs:CreateLogGroup", "Resource": "*"},
        {
            "Effect": "Allow",
            "Action": ["logs:CreateLogStream", "logs:PutRetentionPolicy"],
            "Resource": "arn:aws:logs:{}:{}:log-group:/greengrass/*".format(
                current_region, current_account_id
            ),
        },
        {
            "Effect": "Allow",
            "Action": "logs:PutLogEvents",
            "Resource": "arn:aws:logs:{}:{}:log-group:/greengrass/*:log-stream:*".format(
                current_region, current_account_id
            ),
        },
    ],
}

group_role_arn = create_greengrass_group_role(
    GroupRoleName=GROUP_PREFIX + "GreengrassGroupRole_" + uniqueid(),
    GroupRolePolicy=group_role_policy,
    iam=iam_client,
)

# Associate the role to the Greengrass group
resp = gg_client.associate_role_to_group(GroupId=group["Id"], RoleArn=group_role_arn)

# Print result of API call
resp.pop("ResponseMetadata") and resp

NameError: name 'current_region' is not defined

# Part 3: CoreDefinition, CoreDefinitionVersion and related IoT Thing, certificate, keys and policy
  


### Create the set of private key, public key and certificate and store them 


For the  initial configuration of AWS IoT Greengrass core the following files need to be placed on the edge device (in case of this workshop we are using an EC2 instance to simulate an edge device):
```
/greengrass/certs/<name>.pem => Certificate of the Core device 
/greengrass/certs/<name>.private.key => Private key of the Core device
/greengrass/certs/<name>.public.key  => Public key of the Core device
```    

In the next step the 3 above mentioned files will be generated and stored locally in the directory with the name "greengrassconfig-<Unique id>". In the later steps we will transfer these files to the EC2 instance into `/greengrass/certs`


In [ ]:
# Call Boto3 API to generate certificate, public key and private key
core_thing_keys_cert = iot_client.create_keys_and_certificate(setAsActive=True)

# Store certificate, public key and private key
filename_privatekey = GROUP_NAME + ".private.key"
filename_publickey = GROUP_NAME + ".public.key"
filename_certificate = GROUP_NAME + ".pem"

save_string_to_file(
    ggcore_config_directory_cert + "/" + filename_certificate,
    core_thing_keys_cert["certificatePem"],
)

# Please note: storing private keys on a local file system is generally insecure and is done in 
# this code sample only for demonstrational purposes assuming prototypic usage. It should be only used in 
# prototyping environments after detailed risk analysis and never used in production environments.
#
# Please consult "Hardware security"  document 
# at https://docs.aws.amazon.com/greengrass/latest/developerguide/hardware-security.html
# for best practicess on secure storage and offloading of private keys for Greengrass Core devices.
save_string_to_file(
    ggcore_config_directory_cert + "/" + filename_privatekey,
    core_thing_keys_cert["keyPair"]["PrivateKey"],
)
save_string_to_file(
    ggcore_config_directory_cert + "/" + filename_publickey,
    core_thing_keys_cert["keyPair"]["PublicKey"],
)

In [ ]:
!ls -ls $ggcore_config_directory_cert

### Create Core Thing and attach certificate to the Core Thing  

In [ ]:
core_thing_name = GROUP_PREFIX + "Core_" + uniqueid()
core_thing = iot_client.create_thing(thingName=core_thing_name)
iot_client.attach_thing_principal(
    thingName=core_thing["thingName"], principal=core_thing_keys_cert["certificateArn"]
)

core_thing.pop("ResponseMetadata") and core_thing

### Create new policy for the Core Thing and attach that policy to the certificate  

The policy is only for demonstration purposes and should be reviewed and adjusted before further use in other 
use cases or applications.


In [2]:
# Define policy content.
minimal_core_device_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {"Effect": "Allow", "Action": ["iot:Connect"], "Resource": ["*"]},
        {
            "Effect": "Allow",
            "Action": ["iot:Publish", "iot:Receive"],
            "Resource": [
                "arn:aws:iot:{}:{}:topic/$aws/things/{}".format(
                    current_region, current_account_id, core_thing_name + "*"
                ),
                "arn:aws:iot:{}:{}:topic/dt/coffeemonitor/machines".format( current_region, current_account_id ) 
            ],
        },
        {
            "Effect": "Allow",
            "Action": ["iot:Subscribe"],
            "Resource": [
                "arn:aws:iot:{}:{}:topicfilter/$aws/things/{}".format(
                    current_region, current_account_id, core_thing_name + "*"
                )
            ],
        },
        {
            "Effect": "Allow",
            "Action": [
                "iot:GetThingShadow",
                "iot:UpdateThingShadow",
                "iot:DeleteThingShadow",
            ],
            "Resource": [
                "arn:aws:iot:{}:{}:thing/{}".format(
                    current_region, current_account_id, core_thing_name + "*"
                )
            ],
        },
        {
            "Effect": "Allow",
            "Action": ["greengrass:AssumeRoleForGroup", "greengrass:CreateCertificate"],
            "Resource": ["*"],
        },
        {
            "Effect": "Allow",
            "Action": ["greengrass:GetDeployment"],
            "Resource": [
                "arn:aws:greengrass:{}:{}:/greengrass/groups/{}/deployments/*".format(
                    current_region, current_account_id, group_id
                )
            ],
        },
        {
            "Effect": "Allow",
            "Action": ["greengrass:GetDeploymentArtifacts"],
            "Resource": [
                "arn:aws:greengrass:{}:{}:/greengrass/groups/{}/deployments/*".format(
                    current_region, current_account_id, group_id
                )
            ],
        },
        {
            "Effect": "Allow",
            "Action": ["greengrass:UpdateCoreDeploymentStatus"],
            "Resource": [
                "arn:aws:greengrass:{}:{}:/greengrass/groups/{}/deployments/*/cores/arn%3Aaws%3Aiot%3A{}%3A{}%3Athing%2F{}".format(
                    current_region,
                    current_account_id,
                    group_id,
                    current_region,
                    current_account_id,
                    core_thing_name,
                )
            ],
        },
        {
            "Effect": "Allow",
            "Action": [
                "greengrass:GetConnectivityInfo",
                "greengrass:UpdateConnectivityInfo",
            ],
            "Resource": [
                "arn:aws:iot:{}:{}:thing/{}".format(
                    current_region, current_account_id, core_thing_name
                )
            ],
        },
    ],
}

NameError: name 'current_region' is not defined

In [ ]:
# Create new policy
policy = iot_client.create_policy(
    policyName=GROUP_PREFIX + "CorePolicy_" + uniqueid(),
    policyDocument=json.dumps(minimal_core_device_policy),
)

# Attach certificate to the policy
iot_client.attach_principal_policy(
    policyName=policy["policyName"], principal=core_thing_keys_cert["certificateArn"]
)

policy.pop("ResponseMetadata") and policy.pop("policyDocument") and policy

### Create a CoreDefinition and CoreDefinitionVersion

In [ ]:
core_definition = gg_client.create_core_definition(
    InitialVersion={
        "Cores": [
            {
                "Id": core_thing["thingName"],
                "ThingArn": core_thing["thingArn"],
                "CertificateArn": core_thing_keys_cert["certificateArn"],
                "SyncShadow": False,
            }
        ]
    },
    Name=GROUP_PREFIX + uniqueid(),
)

core_definition.pop("ResponseMetadata") and core_definition

# Part 4: DeviceDefinition and DeviceDefinitionVersion and related IoT Things, certificates, keys and polices

What will happen in this part:    
1. Define device names  
2. Create IoT policies with minimal permissions  
3. Create key material and certificate , store them into local directory  
4. Create Things in IoT registry, attach certificate to each Thing, attach IoT policies to each Thing's certificate  
5. Create DeviceDefinition and DeviceDefinitionVersion  

### Define device names

In [ ]:
NUMBER_OF_DEVICES = 2
device_thing_names = []

for device_id in range(1, NUMBER_OF_DEVICES + 1, 1):
    device_thing_names.append("CoffeeMachine" + str(device_id))

log.info("Device Thing names: {}".format(json.dumps(device_thing_names)))

### Create IoT policies with minimal permissions 

In [ ]:
device_policy_names = []


for device_id in range(0, NUMBER_OF_DEVICES, 1):

    device_policy_name = (
        "Device" + str(device_id) + "_policy_" + GROUP_PREFIX + "_" + uniqueid()
    )
    # Device policy will contain the minimal persmissions, neccessary for a Group discovery process
    device_policy_content = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": ["greengrass:Discover"],
                "Resource": [
                    "arn:aws:iot:{}:{}:thing/{}".format(
                        current_region,
                        current_account_id,
                        device_thing_names[device_id],
                    )
                ],
            }
        ],
    }

    iot_client.create_policy(
        policyName=device_policy_name, policyDocument=json.dumps(device_policy_content),
    )

    device_policy_names.append(device_policy_name)

device_policy_names

### Create key material and certificate , store them into local directory

In [ ]:
device_key_certs, device_simulator_commands = [], []

for device_id in range(0, NUMBER_OF_DEVICES, 1):
    # Call Boto3 API to generate certificate, public key and private key
    device_keys_cert = iot_client.create_keys_and_certificate(setAsActive=True)

    # Store certificate, public key and private key
    filename_privatekey_device = "CoffeeMachine_{}.key".format(device_id + 1)
    filename_publickey_device = "CoffeeMachine_{}.public.key".format(device_id + 1)
    filename_certificate_device = "CoffeeMachine_{}.pem".format(device_id + 1)
    save_string_to_file(
        ggad_config_directory_cert + "/" + filename_certificate_device,
        device_keys_cert["certificatePem"],
    )

    # Please note: storing private keys on a local file system is generally insecure and is done in 
    # this code sample only for demonstrational purposes assuming prototypic usage. It should be only used in 
    # prototyping environments after detailed risk analysis and never used in production environments.
    save_string_to_file(
        ggad_config_directory_cert + "/" + filename_privatekey_device,
        device_keys_cert["keyPair"]["PrivateKey"],
    )
    save_string_to_file(
        ggad_config_directory_cert + "/" + filename_publickey_device,
        device_keys_cert["keyPair"]["PublicKey"],
    )

    device_simulator_commands.append(
        "./coffemachine_simulator.sh {} {} {} {}".format(
            device_thing_names[device_id],
            "certs/" + filename_certificate_device,
            "certs/" + filename_privatekey_device,
            device_id + 1,
        )
    )

    device_key_certs.append(device_keys_cert)

!ls -d $ggad_config_directory_cert/*

### Create Things in IoT registry, attach certificate to each Thing, attach IoT policies to each Thing's certificate

In [ ]:
device_things = []
for device_id in range(0, NUMBER_OF_DEVICES, 1):

    device_thing = iot_client.create_thing(thingName=device_thing_names[device_id])
    device_thing.pop("ResponseMetadata") and print(device_thing)
    device_thing_principal = iot_client.attach_thing_principal(
        thingName=device_thing_names[device_id],
        principal=device_key_certs[device_id]["certificateArn"],
    )
    iot_client.attach_principal_policy(
        policyName=device_policy_names[device_id],
        principal=device_key_certs[device_id]["certificateArn"],
    )
    device_things.append(device_thing)

### Create DeviceDefinition and DeviceDefinitionVersion

In [ ]:
device_definition_version_entries = []
for device_id in range(0, NUMBER_OF_DEVICES, 1):
    device_definition_version_entries.append(
        {
            "CertificateArn": device_key_certs[device_id]["certificateArn"],
            "Id": "device{}".format(device_id),
            "SyncShadow": False,
            "ThingArn": device_things[device_id]["thingArn"],
        }
    )


device_definition = gg_client.create_device_definition(
    InitialVersion={"Devices": device_definition_version_entries},
    Name=GROUP_PREFIX + uniqueid(),
)

device_definition.pop("ResponseMetadata") and device_definition

# Part 5: FunctionDefinition and FunctionDefinitionVersion and related AWS Lambda function

### Install AWS Lambda packages

For the demonstration purposes we will use AWS SDK to create an AWS Lambda function to be deployed on a Greegrass device. For the production scenarios please consider using AWS CloudFormation and AWS SDK.

### Create a file ./build/lambda.zip from directory 'coffeemachine_telemetryprocessor/src'

In [ ]:
!cd "../coffeemachine_telemetryprocessor/src" && pip3 install -r requirements.txt -t .
zipfile_path = "build/lambda.zip"
lambda_code_source_path = "../coffeemachine_telemetryprocessor/src"
!mkdir -p "./build" && cd $lambda_code_source_path && zip -r ../../notebooks/$zipfile_path * >/dev/null
!ls -la $zipfile_path

### Create a new AWS Lambda function with alias "demo" 

In [ ]:
LAMBDA_FUNCTION_NAME = "TelemetryProcessor_" + uniqueid()
LAMBDA_ROLE_NAME = "TelemetryProcessorRole_" + uniqueid()

lambda_function_arn_fullqualified = create_lambda_function(
    FunctionName=LAMBDA_FUNCTION_NAME,
    RoleName=LAMBDA_ROLE_NAME,
    CodeZipFilePath=zipfile_path,
    FunctionAliasName="demo",
    iam=iam_client,
    log=log,
    lambda_client=lambda_client,
)

### FunctionDefinition and FunctionDefinitionVersion 

In [ ]:
function_definition = gg_client.create_function_definition(
    InitialVersion={
        "DefaultConfig": {"Execution": {"IsolationMode": "GreengrassContainer"}},
        "Functions": [
            # AWS Lambda function for local visualization of coffee consumption
            {
                "FunctionArn": lambda_function_arn_fullqualified,
                "FunctionConfiguration": {
                    "EncodingType": "json",
                    "MemorySize": 64 * 1024,
                    "Pinned": True,
                    "Timeout": 20,
                },
                "Id": str(uuid.uuid4()),
            }, 
            # You can configure AWS IoT Greengrass to enable automatic discovery of your Greengrass core 
            # using the IPDetector system Lambda function. You can also enable this feature by choosing Automatic                    # detection when you deploy your group from the console for the first time, or from the 
            # group's Settings page in the console at any time. 
            # Please consult the chapter "Activate automatic IP detection" in
            # https://docs.aws.amazon.com/greengrass/latest/developerguide/gg-core.html for details
            {
                "FunctionArn": "arn:aws:lambda:::function:GGIPDetector:1",
                "FunctionConfiguration": {
                    "Environment": {},
                    "MemorySize": 32768,
                    "Pinned": True,
                    "Timeout": 3
               },
               "Id": str(uuid.uuid4())
            }
        ],
    },
    Name=GROUP_PREFIX + uniqueid(),
)

function_definition.pop("ResponseMetadata") and function_definition

# Part 6: SubscriptionDefinition and SubscriptionDefinitionVersion

Please consider the white paper https://d1.awsstatic.com/whitepapers/Designing_MQTT_Topics_for_AWS_IoT_Core.pdf for best practices on designing MQTT topica for AWS IoT Core

In [ ]:
subscription_definition_version_entries = []

# Allow a communication for each Greengrass aware device (coffee machine) to a Greengrass Device
for device_id in range(0, NUMBER_OF_DEVICES, 1):
    subscription_definition_version_entries.append(
        {
            "Source": device_things[device_id]["thingArn"],
            "Subject": "dt/coffeemonitor/machine/" + str(device_id+1),
            "Target": lambda_function_arn_fullqualified,
            "Id": str(uuid.uuid4()),
        }
    )

# Allow a communication from a Greengrass Device to an AWS IoT cloud
subscription_definition_version_entries.append(
    {
        "Source": lambda_function_arn_fullqualified,
        "Subject": "dt/coffeemonitor/machines",
        "Target": "cloud",
        "Id": str(uuid.uuid4()),
    }
)

subscription_definition = gg_client.create_subscription_definition(
    InitialVersion={"Subscriptions": subscription_definition_version_entries,},
    Name=GROUP_PREFIX + uniqueid(),
)

subscription_definition.pop("ResponseMetadata") and subscription_definition

# Part 7: GroupVersion

In [ ]:
group_version = gg_client.create_group_version(
    GroupId=group["Id"],
    CoreDefinitionVersionArn=core_definition["LatestVersionArn"],
    DeviceDefinitionVersionArn=device_definition["LatestVersionArn"],
    FunctionDefinitionVersionArn=function_definition["LatestVersionArn"],
    SubscriptionDefinitionVersionArn=subscription_definition["LatestVersionArn"],
)

group_version.pop("ResponseMetadata") and group_version

# Part 8: Greengrass configuration file config.json

See configuration details in https://docs.aws.amazon.com/greengrass/latest/developerguide/gg-core.html

In [ ]:
# Please note: storing private keys on a local file system is generally insecure and is done in 
# this code sample only for demonstrational purposes assuming prototypic usage. It should be only used in 
# prototyping environments after detailed risk analysis and never used in production environments.

def replace_map(input, map):
    for replacement in map:
        input = input.replace(replacement, map[replacement])
    return input


path_configjsonfile = ggcore_config_directory_jsonconfig + "/config.json"

configjson_template_content = read_string_from_file(FILE_GGCONFIGTEMPLATE)

iot_endpoint = iot_client.describe_endpoint(endpointType="iot:Data-ATS")["endpointAddress"]
core_thing_arn = core_thing["thingArn"]

configjson_content = replace_map(
    configjson_template_content,
    {
        "{{filename_privatekey}}": filename_privatekey,
        "{{filename_publickey}}": filename_publickey,
        "{{filename_cert}}": filename_certificate,
        "{{iot_endpoint}}": iot_endpoint,
        "{{gg_endpoint}}": "greengrass-ats.iot."
        + boto3.session.Session().region_name
        + ".amazonaws.com",
        "{{core_thing_arn}}": core_thing_arn,
        "{{root_ca_file}}": ROOTCAPEM,
    },
)


save_string_to_file(path_configjsonfile, configjson_content)

log.info(
    "Greengrass Core configuration is saved into the file {}".format(
        path_configjsonfile
    )
)

# Part 9: Create an archive with AWS IoT Greengrass Core configuration


**The content of the archive**
```
certs/IoT3Boto3Workshop_<random>.public.key    => Public Key
certs/IoT3Boto3Workshop_<random>.private.key   => Private Key
certs/IoT3Boto3Workshop_<random>.pem           => Certificate
config/config.json                             => AWS IoT Greengrass Core configuration
```

In [ ]:
# Please note: storing private keys on a local file system is generally insecure and is done in 
# this code sample only for demonstrational purposes assuming prototypic usage. It should be only used in 
# prototyping environments after detailed risk analysis and never used in production environments.

gg_core_config_file = os.getcwd() + "/" + uniqueid() + "-gg_core_config" + ".tgz"
!echo "-------- List of files placed into an archive -----------"
! cd $ggcore_config_directory_root && tar cvfz $gg_core_config_file * 
!echo "--------------------------------------------------------"
log.info(
    "An archive with AWS IoT Greengrass Core configuration is saved to file "
    + gg_core_config_file
)

# Part 10: Create an archive with configuration for connected devices containing public keys, private keys and certificates

In [ ]:
# Please note: storing private keys on a local file system is generally insecure and is done in 
# this code sample only for demonstrational purposes assuming prototypic usage. It should be only used in 
# prototyping environments after detailed risk analysis and never used in production environments.

device_config_file = os.getcwd() + "/" + uniqueid() + "-devices_config.tgz"
!echo "-------- List of files placed into an archive -----------"
!cd $ggad_config_directory_root && tar cvfz $device_config_file * 
!echo "--------------------------------------------------------"
log.info(
    "Key material for the Greengrass aware devices is saved to file "
    + device_config_file 
)

#### Transfer the configuration files
Note that you can use the "Download" function of Jupyter by right-clicking on a name of a file.

1. Please securely store the file gg_core_config-< number >.tgz to the Greengrass Device and follow the instructions in an AWS IoT Greengrass documentation on how to configure and start AWS Greengrass. 

2. Please securely store the file device_config-< number >.tgz to a device which you will be using to run the coffee machine simulator script and follow the instructions on https://github.com/aws-samples/aws-iot-greengrass-boto3 in section 'Setting up Greengrass aware devices'

# Part 11: Deploy a Greengrass Group

Please ensure that an AWS IoT Greengrass Core software is configured based on the archive file created in Part 9 and is running before startinga deployment. 

In case of problems, please consider reviewing https://docs.aws.amazon.com/greengrass/latest/developerguide/gg-troubleshooting.html and evaluating the log files of AWS IoT Greengrass core in the directory:

`/greengrass/ggc/var/log/system`



Example log files to consider:
`/greengrass/ggc/var/log/system/runtime.log`

### 11.1 Associate a Greengrass Service role to the account, if necessary
AWS IoT Greengrass needs the service role to access your Lambda functions and AWS IoT resources. This is necessary for deployments to succeed. The role must have at least minimum permissions in the policy 'AWSGreengrassResourceAccessRolePolicy'

Further approach:
- First, we check if a Greengrass service role already associated to the account. 
- If it is associated, no further action will be performed, i.e. we will proceed with existing Greengrass service role
- If it is not associated, we will create a new Greengrass service role and associate it to the account

#### Check if a Greengrass service role already associated to the account. 

In [ ]:
is_gg_service_role_associated_to_account = False
ggservice_role_arn = None

try:
    resp = gg_client.get_service_role_for_account()
except gg_client.exceptions.ClientError:
    print("No Greengrass service is associated to account yet.")
    is_gg_service_role_associated_to_account = False
else:
    is_gg_service_role_associated_to_account = True
    ggservice_role_arn = resp["RoleArn"]
    print(
        "Greengrass service role is associated to account, role ARN {}".format(
            ggservice_role_arn
        )
    )

#### If no Greengrass Service role is associated, create a new role "IoTWorkshop_GreengrassServiceRole" if it does not exist yet

In [ ]:
# Create a new Greengrass service role if neccessary
if not is_gg_service_role_associated_to_account:
    ggservice_role_name = "IoTWorkshop_GreengrassServiceRole"

    if not role_exists(iam_client, ggservice_role_name):
        # Define the assume role policy
        ggservicerole_assume_role_policy = {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Principal": {"Service": "greengrass.amazonaws.com"},
                    "Action": "sts:AssumeRole",
                }
            ],
        }

        # Create role

        resp = iam_client.create_role(
            RoleName=ggservice_role_name,
            AssumeRolePolicyDocument=json.dumps(ggservicerole_assume_role_policy),
        )

        ggservice_role_arn = resp["Role"]["Arn"]

        # Attach managed policy to the role
        ARN_AWSGreengrassResourceAccessRolePolicy = (
            "arn:aws:iam::aws:policy/service-role/AWSGreengrassResourceAccessRolePolicy"
        )
        response = iam_client.attach_role_policy(
            RoleName=ggservice_role_name,
            PolicyArn=ARN_AWSGreengrassResourceAccessRolePolicy,
        )
        print(
            "Service role {} created with ARN {}".format(
                ggservice_role_name, ggservice_role_arn
            )
        )

    else:
        resp = iam_client.get_role(RoleName=ggservice_role_name)
        ggservice_role_arn = resp["Role"]["Arn"]
        print(
            "Service role {} already exists with ARN {}. No action was performed.".format(
                ggservice_role_name, ggservice_role_arn
            )
        )

In [ ]:
# Associste a new Greengrass service role to the account if neccessary
if not is_gg_service_role_associated_to_account:
    gg_client.associate_service_role_to_account(RoleArn=ggservice_role_arn)
    print(
        "Greengrass service role {} was associated to the account".format(
            ggservice_role_arn
        )
    )

### 11.2 Start a deployment

In [ ]:
deployment = gg_client.create_deployment(
    DeploymentType="NewDeployment",
    GroupId=group["Id"],
    GroupVersionId=group_version["Version"],
)

deployment_id = deployment["DeploymentId"]

deployment.pop("ResponseMetadata") and deployment

### 11.3 Monitor deployment status and wait for a completion of the deployment

In [ ]:
deployment_finished = False
deployment_timeout = False
deployment_status = ""

# Maxium time to monitor deployment status in seconds
max_monitoring_time = 30

while not deployment_finished and not deployment_timeout:

    response = gg_client.get_deployment_status(DeploymentId=deployment_id, GroupId=group["Id"])
    deployment_status_prev = deployment_status
    deployment_status = response["DeploymentStatus"]
    log.info(
        "Waiting for completion of deployment, currently in status {}".format(
            deployment_status
        )
    )

    if deployment_status != deployment_status_prev:
        log.info(
            "Status of deployment changed: {} ->  {}".format(
                deployment_id, deployment_status_prev, deployment_status
            )
        )

    if deployment_status == "Success" or deployment_status == "Failure":
        break

    if max_monitoring_time <= 0:
        log.error("Timeout for deployment {}".format(deployment_id))
        break

    time.sleep(5)
    max_monitoring_time = max_monitoring_time - 5

## Part 12: Simulate ingestion from Greengrass aware devices

Please perform the following steps:

1. Clone the repostory https://github.com/aws-samples/aws-iot-greengrass-boto3 on the device you will use to run simulation scripts.

2. Change to the directory `coffeemachine_devicesimulator`:  
 `cd coffeemachine_devicesimulator`

4. Copy and extract the file `<Number>-devices_config.tgz` that was created in part 9:  
  `tar xzvf <Number>-devices_config.tgz`

5. Please execute the commands which are generated below in two separate shell instances and observe the ingestion

6. Open your web browser and point it to port 8081 of your AWS IoT Greengrass Core runtime environment. You should see a dashboard showing coffee consumption. 

In [ ]:
# The item 4 above refers to these commands:
for cmd in device_simulator_commands:
    print(cmd)

## Part 13:Additional information


In [ ]:
log.info ("Greengrass Group name in an AWS Management Console is "+GROUP_NAME) 